In [2]:
# Here are some libraries we will need:

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()  # for plot styling
import numpy as np

In [19]:
#Reading the dataset

data = pd.read_csv ("./data/Facebook_metrics/dataset_Facebook.csv" ,sep= " ", header = 0)

In [20]:
print (data.columns)

Index(['Page', 'total', 'likes;Type;Category;Post', 'Month;Post',
       'Weekday;Post', 'Hour;Paid;Lifetime', 'Post', 'Total', 'Reach;Lifetime',
       'Post.1', 'Total.1', 'Impressions;Lifetime', 'Engaged',
       'Users;Lifetime', 'Post.2', 'Consumers;Lifetime', 'Post.3',
       'Consumptions;Lifetime', 'Post.4', 'Impressions', 'by', 'people', 'who',
       'have', 'liked', 'your', 'Page;Lifetime', 'Post.5', 'reach', 'by.1',
       'people.1', 'who.1', 'like', 'your.1', 'Page;Lifetime.1', 'People',
       'who.2', 'have.1', 'liked.1', 'your.2', 'Page.1', 'and', 'engaged',
       'with', 'your.3', 'post;comment;like;share;Total', 'Interactions'],
      dtype='object')


In [21]:
data.shape

(500, 47)